### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import re

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv(".env")

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [4]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
nyt_url = f"{url}fq={filter_query}&f1={field_list}&begin_date={begin_date}&end_date={end_date}&sort={sort}&api-key={nyt_api_key}"

In [5]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for i in range(20):
    
    # create query with a page number
    # API results show 10 articles at a time
    page = f"&page={i}"
    response = requests.get(nyt_url + page).json()

     # a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    try:
        
        for doc in response["response"]["docs"]:
            reviews_list.append(doc)
            
        print(f"Checked page {i}")       

    except:
        print(f"No doc found for page {i} ")
        break    


Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [6]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
data = json.dumps(response["response"]["docs"][5], indent=4)
print(data)

{
    "abstract": "This moody romance stars Tatiana Maslany (\u201cOrphan Black\u201d) as a bipolar painter in love with a brooding cabby (Tom Cullen of \u201cDownton Abbey\u201d).",
    "web_url": "https://www.nytimes.com/2017/03/09/movies/the-other-half-review-tatiana-maslany.html",
    "snippet": "This moody romance stars Tatiana Maslany (\u201cOrphan Black\u201d) as a bipolar painter in love with a brooding cabby (Tom Cullen of \u201cDownton Abbey\u201d).",
    "lead_paragraph": "Acting prowess is in vast abundance in the moody romance \u201cThe Other Half.\u201d Tatiana Maslany (the dazzling star of \u201cOrphan Black\u201d) plays Emily, a bipolar painter and teacher; Tom Cullen (Lord Gillingham on \u201cDownton Abbey\u201d) portrays Nickie, a brooding cabdriver and musician whose brother vanished almost five years ago. When these two meet, passionate sparks fly, but not without consequences.",
    "print_section": "C",
    "print_page": "9",
    "source": "The New York Times",
  

In [7]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,Two childhood friends navigate cultural differ...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,A glossy lesson in how to pour nontraditional ...,C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,A radiant Virginie Efira stars as a Parisian t...,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,When a woman falls in love in the sensitive Fr...,C,4,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,This moody romance stars Tatiana Maslany (“Orp...,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,Acting prowess is in vast abundance in the moo...,C,9,The New York Times,"[{'rank': 0, 'subtype': 'hpLarge', 'caption': ...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,A nurse travels to the Ottoman Empire on the e...,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,“The Ottoman Lieutenant” is an overwrought nur...,C,10,The New York Times,"[{'rank': 0, 'subtype': 'articleInline', 'capt...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,...,"Review: 

In [8]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df['title'] = reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df.head(5)

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,...,None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,Two childhood friends navigate cultural differ...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,A glossy lesson in how to pour nontraditional ...,C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,A radiant Virginie Efira stars as a Parisian t...,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,When a woman falls in love in the sensitive Fr...,C,4,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,...,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [9]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,...,None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,Two childhood friends navigate cultural differ...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,A glossy lesson in how to pour nontraditional ...,C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,A radiant Virginie Efira stars as a Parisian t...,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,When a woman falls in love in the sensitive Fr...,C,4,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,...,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,This moody romance stars Tatiana Maslany (“Orp...,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,Acting prowess is in vast abundance in the moo...,C,9,The New York Times,"[{'rank': 0, 'subtype': 'hpLarge', 'caption': ...",subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,A nurse travels to the Ottoman Empire on the e...,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,“The Ottoman Lieutenant” is an overwrought nur...,C,10,The New York Times,"[{'rank': 0, 'subtype': 'articleInline', 'capt...",subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,...,None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman 

In [10]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df['title'].to_list() 
titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [11]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [12]:
# Create an empty list to store the results
tmdb_movies_list  = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if (request_counter % 50) == 0:
        time.sleep(1)
    
    # Perform a "GET" request for The Movie Database
    search_response = requests.get(url + title + tmdb_key_string)

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = search_response.json()["results"][0]["id"]
    
        # Make a request for a the full movie details
        details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?"        
        
        # Execute "GET" request with url
        details_response = requests.get(details_url + tmdb_key_string)
        details_data = details_response.json()
        
        # Extract the genre names into a list
        genres = details_data["genres"]

        # Extract the spoken_languages' English name into a list
        spoken_languages = details_data["spoken_languages"]

        # Extract the production_countries' name into a list
        production_countries = details_data["production_countries"]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append(
            {
                "title" : title,
                "original_title" : details_data["original_title"],
                "budget" : details_data["budget"],
                "original_language" : details_data["original_language"],
                "homepage" : details_data["homepage"],
                "overview" : details_data["overview"],
                "popularity" : details_data["popularity"],
                "runtime" : details_data["runtime"],
                "revenue" : details_data["revenue"],
                "release_date" : details_data["release_date"],
                "vote_average" : details_data["vote_average"],
                "vote_count" : details_data["vote_count"],                
                "genres" : genres,
                "spoken_languages" : spoken_languages,
                "production_countries" : production_countries
            }
        )        
        
        # Print out the title that was found
        print(f"The movie {title} was found")
    except:
        print(f"The movie {title} was not found")

    # Add 1 to the request counter
    request_counter += 1

The movie The Attachment Diaries was found
The movie What’s Love Got to Do With It?’ Probably a Lo was not found
The movie You Can Live Forever was found
The movie A Tourist’s Guide to Love was found
The movie Other People’s Children was found
The movie One True Loves was found
The movie The Lost Weekend: A Love Story was found
The movie A Thousand and One was found
The movie Your Place or Mine was found
The movie Love in the Time of Fentanyl was found
The movie Pamela, a Love Story was found
The movie In From the Side was found
The movie After Love was found
The movie Alcarràs was found
The movie Nelly & Nadine was found
The movie Lady Chatterley’s Lover was found
The movie The Sound of Christmas was found
The movie The Inspection was found
The movie Bones and All was found
The movie My Policeman was found
The movie About Fate was found
The movie Waiting for Bojangles was found
The movie I Love My Dad was found
The movie A Love Song was found
The movie Alone Together was found
The mov

In [13]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[0:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 5.486,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            {
                "id": 18,
                "name": "Drama"
            },
            {
                "id": 9648,
                "name": "Mystery"
            },
            {
                "id": 53,
              

In [14]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",5.486,102,0,2021-10-07,3.000,4,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[{'english_name': 'Spanish', 'iso_639_1': 'es'...","[{'iso_3166_1': 'AR', 'name': 'Argentina'}]"
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,15055,2023-03-24,6.700,33,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",24.422,96,0,2023-04-21,6.300,158,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
3,Other People’s Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.306,104,84178,2022-09-21,6.842,184,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'FR', 'name': 'France'}]"
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,14.097,100,37820,2023-04-07,6.406,69,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,The Last Face,The Last Face,0,en,,"Miguel, a heroic Spanish doctor, puts himself ...",25.462,130,0,2017-01-11,5.600,191,"[{'id': 18, 'name': 'Drama'}]","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
150,Lost in Paris,Paris pieds nus,0,fr,,Fiona visits Paris for the first time to assis...,11.222,83,0,2017-01-14,6.000,113,"[{'id': 35, 'name': 'Comedy'}]","[{'english_name': 'French', 'iso_639_1': 'fr',...","[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'is..."
151,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,3.997,103,0,2016-12-02,6.300,24,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]"
152,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",15.217,111,413844,2017-03-10,6.173,246,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","[{'english_name': 'Turkish', 'iso_639_1': 'tr'...","[{'iso_3166_1': 'TR', 'name': 'Turkey'}, {'iso..."


### Merge and Clean the Data for Export

In [15]:
reviews_df.columns

Index(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'print_section',
       'print_page', 'source', 'multimedia', 'keywords', 'pub_date',
       'document_type', 'news_desk', 'section_name', 'type_of_material', '_id',
       'word_count', 'uri', 'headline.main', 'headline.kicker',
       'headline.content_kicker', 'headline.print_headline', 'headline.name',
       'headline.seo', 'headline.sub', 'byline.original', 'byline.person',
       'byline.organization', 'title'],
      dtype='object')

In [16]:
tmdb_df.columns

Index(['title', 'original_title', 'budget', 'original_language', 'homepage',
       'overview', 'popularity', 'runtime', 'revenue', 'release_date',
       'vote_average', 'vote_count', 'genres', 'spoken_languages',
       'production_countries'],
      dtype='object')

In [17]:
# Merge the New York Times reviews and TMDB DataFrames on title
nyt_reviews_tmdb_df = reviews_df.merge(tmdb_df, how='inner', on='title')
nyt_reviews_tmdb_df.head(2)

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,...,"Argentina, 1970s. A desperate young woman goes...",5.486,102,0,2021-10-07,3.0,4,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[{'english_name': 'Spanish', 'iso_639_1': 'es'...","[{'iso_3166_1': 'AR', 'name': 'Argentina'}]"
1,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,15055,2023-03-24,6.7,33,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."


In [18]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genres", "spoken_languages", "production_countries", "multimedia"]

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'", "{", "}"]

# Loop through the list of columns to fix
for col_name in columns_to_fix:
    # Convert the column to type 'str'
    nyt_reviews_tmdb_df[col_name] = nyt_reviews_tmdb_df[col_name].astype('str')

    # Loop through characters to remove
    for char in characters_to_remove:
        nyt_reviews_tmdb_df[col_name] = nyt_reviews_tmdb_df[col_name].str.replace(char, "")

# Display the fixed DataFrame
nyt_reviews_tmdb_df.head()

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"rank: 0, subtype: xlarge, caption: None, credi...",subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,...,"Argentina, 1970s. A desperate young woman goes...",5.486,102,0,2021-10-07,3.000,4,"id: 18, name: Drama, id: 9648, name: Mystery, ...","english_name: Spanish, iso_639_1: es, name: Es...","iso_3166_1: AR, name: Argentina"
1,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"rank: 0, subtype: xlarge, caption: None, credi...",subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,15055,2023-03-24,6.700,33,"id: 18, name: Drama, id: 10749, name: Romance","english_name: English, iso_639_1: en, name: En...","iso_3166_1: CA, name: Canada, iso_3166_1: US, ..."
2,Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,"rank: 0, subtype: xlarge, caption: None, credi...",subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,...,"After an unexpected break up, a travel executi...",24.422,96,0,2023-04-21,6.300,158,"id: 10749, name: Romance, id: 35, name: Comedy","english_name: English, iso_639_1: en, name: En...","iso_3166_1: US, name: United States of America"
3,A radiant Virginie Efira stars as a Parisian t...,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,When a woman falls in love in the sensitive Fr...,C,4,The New York Times,"rank: 0, subtype: xlarge, caption: None, credi...","subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,...,"Rachel loves her life, her students, her frien...",8.306,104,84178,2022-09-21,6.842,184,"id: 18, name: Drama, id: 35, name: Comedy","english_name: English, iso_639_1: en, name: En...","iso_3166_1: FR, name: France"
4,A film adaptation of Taylor Jenkins Reid’s nov...,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,"A woman stands on a dock, staring out at the s...",C,8,The New York Times,"rank: 0, subtype: xlarge, caption: None, credi...","subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,...,Emma and Jesse are living the perfect life tog...,14.097,100,37820,2023-04-07,6.406,69,"id: 10749, name: Romance, id: 18, name: Drama,...","english_name: English, iso_639_1: en, name: En...","iso_3166_1: CZ, name: Czech Republic, iso_3166..."


In [19]:
nyt_reviews_tmdb_df["genres"].dtypes

dtype('O')

In [46]:
# Drop "byline.person" column
nyt_reviews_tmdb_df.drop("byline.person", axis=1, inplace=True)

In [50]:
print(nyt_reviews_tmdb_df.columns)

Index(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'print_section',
       'print_page', 'source', 'multimedia', 'keywords', 'pub_date',
       'document_type', 'news_desk', 'section_name', 'type_of_material', '_id',
       'word_count', 'uri', 'headline.main', 'headline.kicker',
       'headline.content_kicker', 'headline.print_headline', 'headline.name',
       'headline.seo', 'headline.sub', 'byline.original',
       'byline.organization', 'title', 'original_title', 'budget',
       'original_language', 'homepage', 'overview', 'popularity', 'runtime',
       'revenue', 'release_date', 'vote_average', 'vote_count', 'genres',
       'spoken_languages', 'production_countries'],
      dtype='object')


In [52]:
# Delete duplicate rows and reset index
clean_nyt_reviews_tmdb_df = nyt_reviews_tmdb_df.drop_duplicates().reset_index()
clean_nyt_reviews_tmdb_df

,index,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"rank: 0, subtype: xlarge, caption: None, credi...",subject: Movies;creative_works: The Attachment...,...,"Argentina, 1970s. A desperate young woman goes...",5.486,102,0,2021-10-07,3.000,4,"id: 18, name: Drama, id: 9648, name: Mystery, ...","english_name: Spanish, iso_639_1: es, name: Es...","iso_3166_1: AR, name: Argentina"
1,1,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"rank: 0, subtype: xlarge, caption: None, credi...",subject: Movies;creative_works: You Can Live F...,...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,15055,2023-03-24,6.700,33,"id: 18, name: Drama, id: 10749, name: Romance","english_name: English, iso_639_1: en, name: En...","iso_3166_1: CA, name: Canada, iso_3166_1: US, ..."
2,2,Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,"rank: 0, subtype: xlarge, caption: None, credi...",subject: Movies;creative_works: A Tourist's Gu...,...,"After an unexpected break up, a travel executi...",24.422,96,0,2023-04-21,6.300,158,"id: 10749, name: Romance, id: 35, name: Comedy","english_name: English, iso_639_1: en, name: En...","iso_3166_1: US, name: United States of America"
3,3,A radiant Virginie Efira stars as a Parisian t...,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,When a woman falls in love in the sensitive Fr...,C,4,The New York Times,"rank: 0, subtype: xlarge, caption: None, credi...","subject: Movies;persons: Zlotowski, Rebecca;cr...",...,"Rachel loves her life, her students, her frien...",8.306,104,84178,2022-09-21,6.842,184,"id: 18, name: Drama, id: 35, name: Comedy","english_name: English, iso_639_1: en, name: En...","iso_3166_1: FR, name: France"
4,4,A film adaptation of Taylor Jenkins Reid’s nov...,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,"A woman stands on a dock, staring out at the s...",C,8,The New York Times,"rank: 0, subtype: xlarge, caption: None, credi...","subject: Movies;persons: Bracey, Luke (1989- )...",...,Emma and Jesse are living the perfect life tog...,14.097,100,37820,2023-04-07,6.406,69,"id: 10749, name: Romance, id: 18, name: Drama,...","english_name: English, iso_639_1: en, name: En...","iso_3166_1: CZ, name: Czech Republic, iso_3166..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,149,"In war-torn Liberia, Charlize Theron and Javie...",https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",Sean Penn has some significant experience as a...,C,10,The New York Times,"rank: 0, subtype: hpLarge, caption: None, cred...",subject: Movies;creative_works: The Last Face ...,...,"Miguel, a heroic Spanish doctor, puts himself ...",25.462,130,0,2017-01-11,5.600,191,"id: 18, name: Drama","english_name: English, iso_639_1: en, name: En...","iso_3166_1: US, name: United States of America"
150,150,The filmmakers Fiona Gordon and Dominique Abel...,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,"In “Lost in Paris,” the married directors and ...",C,7,The New York Times,"rank: 0, subtype: sfSpan, caption: None, credi...",creative_works: Lost in P

In [53]:
# Export data to CSV without the index
clean_nyt_reviews_tmdb_df.to_csv("output/collected_data.csv", index=False)